In [1]:
%pip install -q pyspark
%pip install -q pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


~~~~sql
SELECT
  co.order_id,
  co.customer_name,
  co.customer_id,
  ad.address,
  ad.state,
  ad.country,
  ad.zip_code
FROM
  customer_orders co
JOIN address_details ad ON co.order_id = ad.order_id
WHERE LENGTH(ad.address) >= 5 AND ad.address NOT IN ('-', '', 'SOME ADDRESS')
ORDER BY co.order_id;
~~~~

<h5 align="justify"> A constulta SQL retorna a combinação de duas tabelas, a tabela que registra pedidos e uma que registra endereços para onde os pedidos foram enviados. Para juntar as tabelas e utilizado o indicador do pedido, que exista nas duas tabelas. Para garantir que ele retorne apenas endereços validos ele filtra a tabela pelo tamanho do campo address da tabela address_details e verifica o campo tem algumas das strings da lista, e ordena pelo indicador do pedido </h5>


In [2]:
import pandas as pd

customer_orders = pd.DataFrame(data={
    "order_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "customer_name": [
        "cliente2", "cliente3", "cliente6", "cliente4", "cliente6", "cliente5", "cliente1", "cliente6", "cliente6", "cliente3"
    ],
    "customer_id": [2, 3, 6, 4, 6, 5, 1, 6, 6, 3],
})

address_details = pd.DataFrame(data={
    "order_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "address": [
        "Endereço 2", "End", "-", "", "Endereço 5", "Endereço 5", "SOME ADDRESS", "Endereço 6", "Endereço 6", ""
    ],
    "state": ["PA", "", "PI", "", "PA", "PA", "PA", "PR", "PA", ""],
    "country": [
        "BRAZIL", "", "BRAZIL", "", "BRAZIL", "BRAZIL", "BRAZIL", "BRAZIL", "BRAZIL", ""],
    "zip_code": [22, 33, 66, 44, 66, 56, 11, 66, 66, 33]
})


orders_address = pd.merge(
    customer_orders, address_details, on="order_id", how="outer")

filter = (orders_address['address'].str.len() >= 5) \
          & (~orders_address['address'].isin(['-', '', 'SOME ADDRESS']))

orders_address = orders_address[filter] \
  .sort_values('order_id')

orders_address

,order_id,customer_name,customer_id,address,state,country,zip_code
0,1,cliente2,2,Endereço 2,PA,BRAZIL,22
4,5,cliente6,6,Endereço 5,PA,BRAZIL,66
5,6,cliente5,5,Endereço 5,PA,BRAZIL,56
7,8,cliente6,6,Endereço 6,PR,BRAZIL,66
8,9,cliente6,6,Endereço 6,PA,BRAZIL,66


In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, length

spark = SparkSession.builder \
    .master("local[1]") \
    .appName("spark_example") \
    .getOrCreate()

customer_orders_spark=spark.createDataFrame(customer_orders)
address_details_spark=spark.createDataFrame(address_details)

orders_address_spark = customer_orders_spark.join(
    address_details_spark, on="order_id", how="outer"
)

orders_address_spark = orders_address_spark.where(
    (length(col('address')) >= 5) & (~col('address').isin(['-', '', 'SOME ADDRESS']))
).orderBy("order_id")

orders_address_spark.show()

+--------+-------------+-----------+----------+-----+-------+--------+
|order_id|customer_name|customer_id|   address|state|country|zip_code|
+--------+-------------+-----------+----------+-----+-------+--------+
|       1|     cliente2|          2|Endereço 2|   PA| BRAZIL|      22|
|       5|     cliente6|          6|Endereço 5|   PA| BRAZIL|      66|
|       6|     cliente5|          5|Endereço 5|   PA| BRAZIL|      56|
|       8|     cliente6|          6|Endereço 6|   PR| BRAZIL|      66|
|       9|     cliente6|          6|Endereço 6|   PA| BRAZIL|      66|
+--------+-------------+-----------+----------+-----+-------+--------+

